In [1]:
%matplotlib inline
import torch
import torch.nn.functional as F
import os, sys
from pathlib import Path
from TCN.mnist_pixel.utils import data_generator
from TCN.mnist_pixel.model import TCN
import numpy as np
import matplotlib.pyplot as plt
import pickle
from IPython import display
import ipywidgets as widgets
from PIL import Image
import io
import cv2
import time

from torch2trt import torch2trt

In [3]:
model_path = Path('./TCN/mnist_pixel/models')
data_path = Path('./TCN/mnist_pixel/data/mnist')
model_name = 'aug_k7l6'
batch_size = 1
in_channels = 1
n_classes = 10

args = pickle.load(open(model_path / (model_name+'_args.pkl'), 'rb'))
print(args)
channel_sizes = [args.nhid] * args.levels

print(model_path / (model_name+'.pt'))

_, test_loader = data_generator(data_path, batch_size)
model = TCN(in_channels, n_classes, channel_sizes, kernel_size=args.ksize, inference=True)
model.load_state_dict(torch.load(model_path / (model_name+'.pt')), strict=False)
model.eval()

print(model.receptive_field)

if torch.cuda.is_available():
    model.cuda()

Namespace(batch_size=64, clip=-1, cuda=True, dropout=0.05, epochs=50, ksize=7, levels=6, log_interval=100, lr=0.002, modelname='aug_k7l6', nhid=25, optim='Adam', permute=False, savedir=PosixPath('models'), savemodel=True, seed=-1, seq_augment=True)
TCN/mnist_pixel/models/aug_k7l6.pt
757


In [7]:
x = torch.tensor([[[0.0]]], dtype=torch.float).cuda()
model_traced = torch.jit.trace(model, x)
model_trt = torch2trt(model_traced, x)

/usr/local/lib/python3.8/dist-packages/torch/jit/_trace.py:977: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
With rtol=1e-05 and atol=1e-05, found 10 element(s) (out of 10) whose difference(s) exceeded the margin of error (including 0 nan comparisons). The greatest difference was 0.013053178787231445 (-2.277338743209839 vs. -2.2642855644226074), which occurred at index (0, 4).
  _check_trace(


RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
  File "/runfa/shivb/TCN/TCN/tcn_inference.py", line 62, in forward
    def forward(self, x):
        # assert x.size() == self.cache[:,:,0:1].size()
        cache_update = torch.cat((self.cache[:,:,1:], x.detach()), dim=2)
                       ~~~~~~~~~ <--- HERE
        self.cache[:,:,:] = cache_update
        return self.cache
RuntimeError: Tensors must have same number of dimensions: got 2 and 3
